In [ ]:
from __future__ import annotations
from titanic_lab.paths import (
    ROOT,
    TRAIN_CSV,
)  # and whatever common files that were defined in the src/paths.py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
df = pd.read_csv(TRAIN_CSV)
#with pd.option_context(
    #"display.max_rows", None,
    #"display.max_columns", None,   # show all columns too
    #"display.width", None,         # don't wrap awkwardly
    #"display.max_colwidth", None   # don't truncate long strings
print(df.head())   # or display(df) in Jupyter



   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


I think I can extract title from name and find counts

In [37]:
df["Title_raw"] = (
    df["Name"]
      .str.extract(r",\s*([^\.]+)\.", expand=False)  # text between ',' and '.'
      .str.strip()
)

title_map = {
    # common keepers
    "Mr": "Mr",
    "Mrs": "Mrs",
    "Miss": "Miss",
    "Master": "Master",
    "Ms": "Miss",

    # spellings / languages → common
    "Mlle": "Miss",
    "Mme": "Mrs",

    # professional / clergy / rank
    "Dr": "Dr",
    "Rev": "Clergy",
    "Col": "Military",
    "Major": "Military",
    "Capt": "Military",

    # nobility / honorifics
    "Sir": "Royalty",
    "Lady": "Royalty",
    "Countess": "Royalty",
    "Don": "Royalty",
    "Dona": "Royalty",
    "Jonkheer": "Royalty",
}

df["Title"] = df["Title_raw"].map(title_map).fillna(df["Title_raw"])  # default to itself if unseen

with pd.option_context(
    "display.max_rows", None,
    "display.max_columns", None,
    "display.width", None,
    "display.max_colwidth", None,
):
    print(df["Title"])

0                Mr
1               Mrs
2              Miss
3               Mrs
4                Mr
5                Mr
6                Mr
7            Master
8               Mrs
9               Mrs
10             Miss
11             Miss
12               Mr
13               Mr
14             Miss
15              Mrs
16           Master
17               Mr
18              Mrs
19              Mrs
20               Mr
21               Mr
22             Miss
23               Mr
24             Miss
25              Mrs
26               Mr
27               Mr
28             Miss
29               Mr
30          Royalty
31              Mrs
32             Miss
33               Mr
34               Mr
35               Mr
36               Mr
37               Mr
38             Miss
39             Miss
40              Mrs
41              Mrs
42               Mr
43             Miss
44             Miss
45               Mr
46               Mr
47             Miss
48               Mr
49              Mrs


In [23]:
print("Shape:", df.shape)  # expect (891, 12)
print("\nInfo():")
df.info()


Shape: (891, 12)

Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
num_desc = df.describe()
print(num_desc)


In [ ]:
cat_desc = df.describe(include=["object", "category", "boolean"])
print(cat_desc)


In [ ]:
na = df.isna().sum().sort_values(ascending=False)
print("\nMissing values per column:")
print(na)


# Using the df["Age"] column with the .notna() method chained to the sum method takes all of the values
# This counts the number of non-missing values in the "Age" column.
# .notna() returns a boolean Series (True for present values), and .sum() counts the Trues.


In [ ]:
n_non_missing_age = df["Age"].notna().sum()
print(f"\nNon-missing Age count: {n_non_missing_age} (expect 714)")


In [ ]:


def plot_rate(series: pd.Series, title: str) -> None:
    ax = series.sort_values(ascending=False).plot(kind="bar")
    ax.set_ylim(0, 1)
    ax.set_ylabel("Survival rate")
    ax.set_title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
df["HasCabin"] = df["Cabin"].notna()
plot_rate(df.groupby("HasCabin")["Survived"].mean(), "Survival by Cabin presence")


In [ ]:
print("\nSurvived value counts (0/1):")
print(df["Survived"].value_counts(dropna=False).sort_index())


In [ ]:
rate_by_sex = df.groupby("Sex")["Survived"].mean()
plot_rate(rate_by_sex, "Survival rate by Sex")


In [ ]:
rate_by_pclass = df.groupby("Pclass")["Survived"].mean()
plot_rate(rate_by_pclass, "Survival rate by Pclass")


In [ ]:
rate_by_embarked = df.groupby("Embarked", dropna=False)["Survived"].mean()
plot_rate(rate_by_embarked, "Survival rate by Embarked")


In [ ]:
pivot_sex_pclass = pd.pivot_table(
    df, index="Sex", columns="Pclass", values="Survived", aggfunc="mean"
)
print("\nPivot (Sex x Pclass) = mean survival:")
print(pivot_sex_pclass)


In [ ]:
ax = pivot_sex_pclass.plot(kind="bar")
ax.set_ylim(0, 1)
ax.set_ylabel("Survival rate")
ax.set_title("Survival rate by Sex and Pclass")
plt.legend(title="Pclass")
plt.tight_layout()
plt.show()


In [ ]:
# Age distribution (note: missing values exist)
ax = df["Age"].dropna().plot(kind="hist", bins=30)
ax.set_title("Age distribution (train)")
ax.set_xlabel("Age")
plt.tight_layout()
plt.show()

In [ ]:
# Fare is skewed; look at both linear and log scales
ax = df["Fare"].dropna().plot(kind="hist", bins=40)
ax.set_title("Fare distribution (linear scale)")
ax.set_xlabel("Fare")
plt.tight_layout()
plt.show()

In [ ]:
ax = df["Fare"].dropna().plot(kind="hist", bins=40, log=True)
ax.set_title("Fare distribution (log scale)")
ax.set_xlabel("Fare")
plt.tight_layout()
plt.show()

In [ ]:

# quick sanity check on what boxplot should look like
out = (
    df[["Survived", "Age"]]
    .dropna()
    .groupby("Survived")["Age"]
    .agg(
        median="median",
        mean="mean",
        q1=lambda s: s.quantile(0.25),
        q3=lambda s: s.quantile(0.75),
        n="size",
    )
)
print(out)

fig, ax = plt.subplots(figsize=(6, 4))
df.boxplot(column="Age", by="Survived", ax=ax)

# Pandas adds a figure-level suptitle when you use `by=`—delete it:
fig.suptitle("")

ax.set_title("Age by Survived")
ax.set_xlabel("Survived")
ax.set_ylabel("Age")
plt.tight_layout()
plt.show()

In [ ]:


out = (
    df[["Survived", "Fare"]]
    .dropna()
    .groupby("Survived")["Fare"]
    .agg(
        median="median",
        mean="mean",
        q1=lambda s: s.quantile(0.25),
        q2=lambda s: s.quantile(0.50),
        q3=lambda s: s.quantile(0.75),
        n="size",
    )
)
print("Summary statistics for Fare by Survived:")
print()
print(out)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
df.boxplot(column="Fare", by="Survived", ax=ax)
ax.set_title("Fare by Survived")
ax.set_xlabel("Survived")
ax.set_ylabel("Fare")
fig.suptitle("")  # remove automatic suptitle
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
df.boxplot(column="Fare", by="Survived", ax=ax)
ax.set_yscale("log")  # <- log axis
ax.set_title("Fare by Survived (log scale)")
ax.set_xlabel("Survived")
ax.set_ylabel("Fare (log scale)")
fig.suptitle("")
plt.tight_layout()
plt.show()


In [ ]:

df2 = df.copy()
df2["Fare_log1p"] = np.log1p(df2["Fare"].clip(lower=0))  # guard against tiny negatives

fig, ax = plt.subplots(figsize=(6, 4))
df2.boxplot(column="Fare_log1p", by="Survived", ax=ax)
ax.set_title("log1p(Fare) by Survived")
ax.set_xlabel("Survived")
ax.set_ylabel("log1p(Fare)")
fig.suptitle("")
plt.tight_layout()
plt.show()


In [ ]:
# %% boxplot for Fare w/ log scle

In [ ]:


def outlier_stats(df, value_col="Fare", group_col="Survived"):
    rows = []
    for g, s in df[[group_col, value_col]].dropna().groupby(group_col)[value_col]:
        s = s.astype(float)
        q1, q3 = s.quantile([0.25, 0.75])
        iqr = q3 - q1
        low_fence = q1 - 1.5 * iqr
        high_fence = q3 + 1.5 * iqr

        n = len(s)
        n_low = int((s < low_fence).sum())
        n_high = int((s > high_fence).sum())
        n_out = n_low + n_high

        rows.append(
            {
                group_col: g,
                "n": n,
                "q1": q1,
                "median": s.median(),
                "q3": q3,
                "iqr": iqr,
                "low_fence": low_fence,
                "high_fence": high_fence,
                # counts
                "n_outliers_low": n_low,
                "n_outliers_high": n_high,
                "n_outliers_total": n_out,
                # percentages
                "pct_in_box": s.between(q1, q3).mean(),  # ~0.50 by definition
                "pct_outliers_low": n_low / n,
                "pct_outliers_high": n_high / n,
                "pct_outliers_total": n_out / n,
                # optional: make counts comparable per-100 passengers
                "outliers_high_per_100": 100 * n_high / n,
            }
        )
    cols_order = [
        group_col,
        "n",
        "q1",
        "median",
        "q3",
        "iqr",
        "low_fence",
        "high_fence",
        "n_outliers_low",
        "n_outliers_high",
        "n_outliers_total",
        "pct_in_box",
        "pct_outliers_low",
        "pct_outliers_high",
        "pct_outliers_total",
        "outliers_high_per_100",
    ]
    return pd.DataFrame(rows)[cols_order]


out = outlier_stats(df, "Fare", "Survived")
print(
    out.assign(
        pct_in_box=lambda d: d.pct_in_box.round(3),
        pct_outliers_low=lambda d: d.pct_outliers_low.round(3),
        pct_outliers_high=lambda d: d.pct_outliers_high.round(3),
        pct_outliers_total=lambda d: d.pct_outliers_total.round(3),
        outliers_high_per_100=lambda d: d.outliers_high_per_100.round(1),
    ).round(2)
)


In [ ]:
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df["FamilyBin"] = pd.cut(
    df["FamilySize"], bins=[0, 1, 2, 4, 8, 20], labels=["1", "2", "3-4", "5-8", "9+"]
)
plot_rate(df.groupby("FamilyBin")["Survived"].mean(), "Survival by Family size")


In [ ]:
OUT = ROOT / "outputs" / "eda"
OUT.mkdir(parents=True, exist_ok=True)
pivot_sex_pclass.to_csv(OUT / "pivot_sex_pclass.csv", index=True)
na.to_csv(OUT / "missingness.csv", header=["n_missing"])
print(f"\nSaved pivot and missingness tables to {OUT}")
